In [1]:
#@title import dependencies
import os
import re
import gc
import json
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.sparse import coo_matrix
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds

np.random.seed(42)

# Dataset Downloading and Analysis

In [2]:
# @title Download dataset from Kaggle
import kagglehub

path = "/root/.cache/kagglehub/datasets/himanshuwagh/spotify-million/versions/1"
if not os.path.exists(path):
  # Download latest version
  path = kagglehub.dataset_download("himanshuwagh/spotify-million")

# contiene le slices del dataset: 1000 slice das 1000 playlist ciascuna
data: str = os.path.join(path, "data")

In [3]:
# @title shuffle slices in a list and pick from them
shuffled_slices = np.array(os.listdir(data))
np.random.shuffle(shuffled_slices)

In [4]:
shuffled_slices[:3]

array(['mpd.slice.528000-528999.json', 'mpd.slice.920000-920999.json',
       'mpd.slice.730000-730999.json'], dtype='<U28')

# SVD Approach

## Dataset Handling

In [5]:
million_df = pd.DataFrame()
num_training_files = 500

# Create an empty list to hold all rows as dictionaries
data_list = []
uri_to_info = dict()

#for i, filename in tqdm(enumerate(sorted(os.listdir(data), key=extract_starting_number)[:num_training_files]), desc="Processing Slices"):
for i, filename in tqdm(enumerate(shuffled_slices[:num_training_files]), desc="Processing Slices", total = num_training_files):
    if filename.startswith("mpd.slice.") and filename.endswith(".json"):
        filepath = os.path.join(data, filename)

        with open(filepath, "r", encoding="utf-8") as jsonfile:
            cur_slice = json.load(jsonfile)

        # for playlist in tqdm(cur_slice["playlists"], desc="Processing playlist..."):
        for playlist in cur_slice["playlists"]:
            playlist_id = playlist["pid"]
            # num_tracks = playlist["num_tracks"]

            # Collect data for the playlist
            for track in playlist["tracks"]:
                data_list.append({
                    "playlist": playlist_id,
                    "track": track["track_uri"][14:]  # remove 'spotify:track:'
                })
                if track["track_uri"][14:] not in uri_to_info:
                  uri_to_info[track["track_uri"][14:]] = (track["artist_name"], track["track_name"])

    # update every 30 files for speedup
    if i%30 == 0:
        new_data = pd.DataFrame(data_list)
        data_list.clear()
        million_df = pd.concat([million_df, new_data], ignore_index=True)

# Convert the list of dictionaries into a DataFrame in one go
# dumb_dataset = pd.DataFrame(data_list)
new_data = pd.DataFrame(data_list)
data_list = []
million_df = pd.concat([million_df, new_data], ignore_index=True)

million_df["playlist"] = million_df["playlist"].astype("int32")

Processing Slices: 100%|██████████| 500/500 [05:32<00:00,  1.50it/s]


In [6]:
million_df.shape

(33170567, 2)

In [7]:
million_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33170567 entries, 0 to 33170566
Data columns (total 2 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   playlist  int32 
 1   track     object
dtypes: int32(1), object(1)
memory usage: 379.6+ MB


In [8]:
million_df.head()

,playlist,track
0,528000,5mmgfPAMIFIhlP2VneJc0G
1,528000,40riOy7x9W7GXjyGp4pjAv
2,528000,4efoEY8iDBzUqitjmNDhpN
3,528000,0NqQmmLEN9rlnkh2JW0UIs
4,528000,1MQCTOWVfy4PcuBXkBsHVB


In [9]:
# Count how many playlists each track appears in
track_frequency = million_df.groupby("track")["playlist"].nunique()
total_songs = len(track_frequency)

# Convert to a DataFrame for easier handling
track_frequency_df = track_frequency.reset_index().rename(columns={"playlist": "playlist_count"})

# Total number of playlists
# total_playlists = million_df["playlist_id"].nunique()
total_playlists = 1000*num_training_files

# threshold
threshold = int(total_playlists * 0.00005)

# Filter tracks that appear in at least the threshold number of playlists
popular_tracks = track_frequency_df[track_frequency_df["playlist_count"] >= threshold]
partial = len(popular_tracks)

# Extract popular track IDs
popular_track_ids = popular_tracks["track"].tolist()

# Filter the original dataset
filtered_df = million_df[million_df["track"].isin(popular_track_ids)]


In [10]:
print("Minumum number of playlists: ", threshold)
ratio = partial / total_songs
print("Total songs: ", total_songs)
print(f"Songs that appear in at least {threshold} playlists: ", partial)
print(f"Ratio: {ratio:.2f}")

Minumum number of playlist:  25
Total songs:  1608601
Songs that appear in at least 25 playlists:  119003
Ratio: 0.07


In [11]:
print("Songs with most appearences in playlists: ")
for uri, playlist in zip(track_frequency_df.sort_values("playlist_count", ascending=False).head()["track"], track_frequency_df.sort_values("playlist_count", ascending=False).head()["playlist_count"]):
  artist, name = uri_to_info[uri]
  print(f"{artist} - {name} (Appears in {playlist} playlists, {playlist/total_playlists:.2f}% of total playlist in the training set)")

Songs with most appearences in playlists: 
Kendrick Lamar - HUMBLE. (Appears in 22593 playlists, 0.05% of total playlist in the training set)
Drake - One Dance (Appears in 20669 playlists, 0.04% of total playlist in the training set)
DRAM - Broccoli (feat. Lil Yachty) (Appears in 20275 playlists, 0.04% of total playlist in the training set)
The Chainsmokers - Closer (Appears in 20236 playlists, 0.04% of total playlist in the training set)
Post Malone - Congratulations (Appears in 19674 playlists, 0.04% of total playlist in the training set)


In [12]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28469745 entries, 1 to 33170566
Data columns (total 2 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   playlist  int32 
 1   track     object
dtypes: int32(1), object(1)
memory usage: 543.0+ MB


In [13]:
filtered_df.shape

(28469745, 2)

In [14]:
# Make an explicit copy of filtered_df
filtered_df = filtered_df.copy()

# Map playlist_id and track_id to numerical indices
playlist_id_to_idx = {id: idx for idx, id in enumerate(filtered_df["playlist"].unique())}
track_uri_to_idx = {uri: idx for idx, uri in enumerate(filtered_df["track"].unique())}

filtered_df["playlist_idx"] = filtered_df["playlist"].map(playlist_id_to_idx)
filtered_df["track_idx"] = filtered_df["track"].map(track_uri_to_idx)

# Create COO matrix
rows = filtered_df["playlist_idx"]
cols = filtered_df["track_idx"]
data_list = np.ones(len(filtered_df))  # All entries are 1 since a track belongs to a playlist

coo_rating_matrix = coo_matrix((data_list, (rows, cols)), shape=(len(playlist_id_to_idx), len(track_uri_to_idx)))
print(coo_rating_matrix.shape)  # Output: (485376, 18857)


(497138, 119003)


In [15]:
# per rendere il codice della funzione di valutazione come quello sopra
num_tracks = coo_rating_matrix.shape[1]
tracks = set(track_uri_to_idx.keys())

In [16]:
json.dump(uri_to_info, open("uri_to_info.json", "w"))
del(uri_to_info)
del(million_df, track_frequency_df, filtered_df)
del(rows, cols, data_list, new_data)
del(popular_tracks, popular_track_ids)
del(playlist_id_to_idx, track_frequency)

gc.collect()

32

In [17]:
gc.collect()

0

## Training

In [18]:
class ScipySVD():
  def __init__(self, n_components, **kwargs):
    self.n_components = n_components
    self.kwargs = kwargs


  def fit(self, X):
    _, _, components_ = svds(X, self.n_components, **self.kwargs)
    self.components_ = components_


  def transform(self ,X):
    return X @ self.components_.T

In [19]:
svd_model = ScipySVD(600, random_state=42)

svd_model.fit(coo_rating_matrix)

## Validation

In [33]:
#@title metrics definition
def precision_at_k(predicted_matrix, ground_truth_matrix, k):
  # Usa argsort per ottenere i top k indici in modo efficiente
  top_k_indices = np.argsort(predicted_matrix, axis=1)[:, ::-1][:, :k]

  # Estrai gli elementi rilevanti nelle prime k raccomandazioni
  relevant_items = ground_truth_matrix[np.arange(ground_truth_matrix.shape[0])[:, None], top_k_indices]

  # Calcola la precisione come il numero di elementi rilevanti diviso k
  precision_scores = np.sum(relevant_items, axis=1) / k

  # Restituisci la precisione media
  return np.mean(precision_scores)


def recall_at_k(predicted_matrix, ground_truth_matrix, k):
    # Get the indices of the top k predictions for each row
    top_k_indices = np.argsort(predicted_matrix, axis=1)[:, -k:][:, ::-1]

    # Gather the relevant items in ground truth corresponding to top k predictions
    relevant_items = ground_truth_matrix[np.arange(ground_truth_matrix.shape[0])[:, None], top_k_indices]

    # Calculate the recall for each playlist
    total_relevant = np.sum(ground_truth_matrix, axis=1)  # Total relevant items per playlist

    # Avoid division by zero: mask rows with no relevant items
    recall_scores = np.sum(relevant_items, axis=1) / np.maximum(total_relevant, 1)

    # Return the mean recall, ignoring rows with no relevant items
    return np.mean(recall_scores[total_relevant > 0])


def mean_reciprocal_rank(predicted_matrix, ground_truth_matrix):
    reciprocal_ranks = []

    # Iterate over each playlist (row in the matrix)
    for pred_row, true_row in zip(predicted_matrix, ground_truth_matrix):
        # Get the indices sorted by predicted scores in descending order
        sorted_indices = np.argsort(pred_row)[::-1]

        # Find the rank of the first relevant item
        found_relevant = False
        for rank, index in enumerate(sorted_indices, start=1):
            if true_row[index] == 1:  # If the item is relevant in the ground truth
                reciprocal_ranks.append(1 / rank)
                found_relevant = True
                break

        # If no relevant items were found, append 0
        if not found_relevant:
            reciprocal_ranks.append(0)

    # Return the mean of the reciprocal ranks
    return np.mean(reciprocal_ranks)



In [31]:
#@title evaluating functions definition
def evaluate_model_k_tracks_removed_df(model, k, num_valid_files=10):
  """
  evaluate model processing a slice of playlists, 200 playlist at time to avoid
  colab cpu overflow
  """

  precision_at_10 = np.zeros(num_valid_files)
  precision_at_5 = np.zeros(num_valid_files)
  precision_at_2 = np.zeros(num_valid_files)
  precision_at_1 = np.zeros(num_valid_files)

  recall_at_10 = np.zeros(num_valid_files)
  recall_at_5 = np.zeros(num_valid_files)
  recall_at_2 = np.zeros(num_valid_files)
  recall_at_1 = np.zeros(num_valid_files)

  mrr = np.zeros(num_valid_files)

  for file_idx, filename in enumerate(shuffled_slices[num_training_files:num_training_files+num_valid_files]):
    correct_playlists = np.zeros((1000, num_tracks))
    p_counter = -1
    if filename.startswith("mpd.slice.") and filename.endswith(".json"):
      filepath = os.path.join(data, filename)

      with open(filepath, "r", encoding="utf-8") as jsonfile:
        cur_slice = json.load(jsonfile)

      for playlist in cur_slice["playlists"]:
        p_counter += 1

        for track in playlist["tracks"]:
          track_uri = track["track_uri"][14:]

          if track_uri in tracks:
            t_idx = track_uri_to_idx[track_uri]

            correct_playlists[p_counter, t_idx] = 1


    incomplete_playlists = np.copy(correct_playlists)

    # Turn exactly k ones to zeros per row
    for row in incomplete_playlists:
      # Get the indices of `1`s in the current row
      one_indices = np.where(row == 1)[0]

      # If there are at least k ones, randomly choose 2 of them
      if len(one_indices) >= k:
        indices_to_zero = np.random.choice(one_indices, size=k, replace=False)
        row[indices_to_zero] = 0


    n_iter = 5

    cur_precision_at_10 = [0 for _ in range(n_iter)]
    cur_precision_at_5 = [0 for _ in range(n_iter)]
    cur_precision_at_2 = [0 for _ in range(n_iter)]
    cur_precision_at_1 = [0 for _ in range(n_iter)]

    cur_recall_at_10 = [0 for _ in range(n_iter)]
    cur_recall_at_5 = [0 for _ in range(n_iter)]
    cur_recall_at_2 = [0 for _ in range(n_iter)]
    cur_recall_at_1 = [0 for _ in range(n_iter)]

    cur_mrr = [0 for _ in range(n_iter)]

    size_batch = 1000 // n_iter

    for iter in range(n_iter):
      P_new = model.transform(incomplete_playlists[size_batch*iter:size_batch*(iter+1), :])

      # Predici la matrice ricostruita per le nuove playlist
      predicted_matrix = np.dot(P_new, model.components_)

      cur_precision_at_10[iter] = precision_at_k(predicted_matrix, correct_playlists[n_iter*iter:n_iter*(iter+1), :], 10)
      cur_precision_at_5[iter] = precision_at_k(predicted_matrix, correct_playlists[n_iter*iter:n_iter*(iter+1), :], 5)
      cur_precision_at_2[iter] = precision_at_k(predicted_matrix, correct_playlists[n_iter*iter:n_iter*(iter+1), :], 2)
      cur_precision_at_1[iter] = precision_at_k(predicted_matrix, correct_playlists[n_iter*iter:n_iter*(iter+1), :], 1)

      cur_recall_at_10[iter] = recall_at_k(predicted_matrix, correct_playlists[n_iter*iter:n_iter*(iter+1), :], 10)
      cur_recall_at_5[iter] = recall_at_k(predicted_matrix, correct_playlists[n_iter*iter:n_iter*(iter+1), :], 5)
      cur_recall_at_2[iter] = recall_at_k(predicted_matrix, correct_playlists[n_iter*iter:n_iter*(iter+1), :], 2)
      cur_recall_at_1[iter] = recall_at_k(predicted_matrix, correct_playlists[n_iter*iter:n_iter*(iter+1), :], 1)

      cur_mrr[iter] = mean_reciprocal_rank(predicted_matrix, correct_playlists[n_iter*iter:n_iter*(iter+1), :])

    precision_at_10[file_idx] = np.mean(cur_precision_at_10)
    precision_at_5[file_idx] = np.mean(cur_precision_at_5)
    precision_at_2[file_idx] = np.mean(cur_precision_at_2)
    precision_at_1[file_idx] = np.mean(cur_precision_at_1)
    recall_at_10[file_idx] = np.mean(cur_recall_at_10)
    recall_at_5[file_idx] = np.mean(cur_recall_at_5)
    recall_at_2[file_idx] = np.mean(cur_recall_at_2)
    recall_at_1[file_idx] = np.mean(cur_recall_at_1)
    mrr[file_idx] = np.mean(cur_mrr)

  print("Precision@10 = ",np.mean(precision_at_10))
  print("Precision@5 = ",np.mean(precision_at_5))
  print("Precision@2 = ",np.mean(precision_at_2))
  print("Precision@1 = ",np.mean(precision_at_1))

  print("Recall@10 = ",np.mean(recall_at_10))
  print("Recall@5 = ",np.mean(recall_at_5))
  print("Recall@2 = ",np.mean(recall_at_2))
  print("Recall@1 = ",np.mean(recall_at_1))

  print("MRR = ", np.mean(mrr))




def evaluate_model_k_tracks_per_playlist(model, k, num_valid_files=10):
  """
  evaluate model processing a slice of playlists, 200 playlist at time to avoid
  colab cpu overflow
  """

  # num_valid_files = 1000 - num_training_files
  precision_at_10 = np.zeros(num_valid_files)
  precision_at_5 = np.zeros(num_valid_files)
  precision_at_2 = np.zeros(num_valid_files)
  precision_at_1 = np.zeros(num_valid_files)

  recall_at_10 = np.zeros(num_valid_files)
  recall_at_5 = np.zeros(num_valid_files)
  recall_at_2 = np.zeros(num_valid_files)
  recall_at_1 = np.zeros(num_valid_files)

  mrr = np.zeros(num_valid_files)

  for file_idx, filename in enumerate(shuffled_slices[num_training_files:num_training_files+num_valid_files]):
    correct_playlists = np.zeros((1000, num_tracks))
    p_counter = -1
    if filename.startswith("mpd.slice.") and filename.endswith(".json"):
      filepath = os.path.join(data, filename)

      with open(filepath, "r", encoding="utf-8") as jsonfile:
        cur_slice = json.load(jsonfile)

      for playlist in cur_slice["playlists"]:
        p_counter += 1

        for track in playlist["tracks"]:
          track_uri = track["track_uri"][14:]

          if track_uri in tracks:
            t_idx = track_uri_to_idx[track_uri]

            correct_playlists[p_counter, t_idx] = 1


    incomplete_playlists = np.copy(correct_playlists)

    for row in incomplete_playlists:
      one_indexes = np.where(row == 1)[0]

      if len(one_indexes) >= k:
        indices_to_zero = np.random.choice(one_indexes, size=(len(one_indexes)-k), replace=False)
        row[indices_to_zero] = 0

    n_iter = 5

    cur_precision_at_10 = [0 for _ in range(n_iter)]
    cur_precision_at_5 = [0 for _ in range(n_iter)]
    cur_precision_at_2 = [0 for _ in range(n_iter)]
    cur_precision_at_1 = [0 for _ in range(n_iter)]

    cur_recall_at_10 = [0 for _ in range(n_iter)]
    cur_recall_at_5 = [0 for _ in range(n_iter)]
    cur_recall_at_2 = [0 for _ in range(n_iter)]
    cur_recall_at_1 = [0 for _ in range(n_iter)]

    cur_mrr = [0 for _ in range(n_iter)]

    size_batch = 1000 // n_iter

    for iter in range(n_iter):
      P_new = model.transform(incomplete_playlists[size_batch*iter:size_batch*(iter+1), :])

      # Predici la matrice ricostruita per le nuove playlist
      predicted_matrix = np.dot(P_new, model.components_)

      cur_precision_at_10[iter] = precision_at_k(predicted_matrix, correct_playlists[n_iter*iter:n_iter*(iter+1), :], 10)
      cur_precision_at_5[iter] = precision_at_k(predicted_matrix, correct_playlists[n_iter*iter:n_iter*(iter+1), :], 5)
      cur_precision_at_2[iter] = precision_at_k(predicted_matrix, correct_playlists[n_iter*iter:n_iter*(iter+1), :], 2)
      cur_precision_at_1[iter] = precision_at_k(predicted_matrix, correct_playlists[n_iter*iter:n_iter*(iter+1), :], 1)

      cur_recall_at_10[iter] = recall_at_k(predicted_matrix, correct_playlists[n_iter*iter:n_iter*(iter+1), :], 10)
      cur_recall_at_5[iter] = recall_at_k(predicted_matrix, correct_playlists[n_iter*iter:n_iter*(iter+1), :], 5)
      cur_recall_at_2[iter] = recall_at_k(predicted_matrix, correct_playlists[n_iter*iter:n_iter*(iter+1), :], 2)
      cur_recall_at_1[iter] = recall_at_k(predicted_matrix, correct_playlists[n_iter*iter:n_iter*(iter+1), :], 1)

      cur_mrr[iter] = mean_reciprocal_rank(predicted_matrix, correct_playlists[n_iter*iter:n_iter*(iter+1), :])

    precision_at_10[file_idx] = np.mean(cur_precision_at_10)
    precision_at_5[file_idx] = np.mean(cur_precision_at_5)
    precision_at_2[file_idx] = np.mean(cur_precision_at_2)
    precision_at_1[file_idx] = np.mean(cur_precision_at_1)
    recall_at_10[file_idx] = np.mean(cur_recall_at_10)
    recall_at_5[file_idx] = np.mean(cur_recall_at_5)
    recall_at_2[file_idx] = np.mean(cur_recall_at_2)
    recall_at_1[file_idx] = np.mean(cur_recall_at_1)
    mrr[file_idx] = np.mean(cur_mrr)

  print("Precision@10 = ",np.mean(precision_at_10))
  print("Precision@5 = ",np.mean(precision_at_5))
  print("Precision@2 = ",np.mean(precision_at_2))
  print("Precision@1 = ",np.mean(precision_at_1))

  print("Recall@10 = ",np.mean(recall_at_10))
  print("Recall@5 = ",np.mean(recall_at_5))
  print("Recall@2 = ",np.mean(recall_at_2))
  print("Recall@1 = ",np.mean(recall_at_1))

  print("MRR = ", np.mean(mrr))

In [ ]:
K1 = [0, 2, 5, 15, 30]
K2 = [2, 5, 15, 30]

for k in K1:
  print(f"\n Validation Metrics Removing {k} songs:")
  evaluate_model_k_tracks_removed_df(svd_model, k, 2)
  print()

print("\n")

for k in K2:
  print(f"\n Validation Metrics Keeping {k} songs per playlist:")
  evaluate_model_k_tracks_per_playlist(svd_model, k, 2)
  print()


# Language Model Approach